In [1]:
!pip install flair

     |████████████████████████████████| 337kB 2.7MB/s 
     |████████████████████████████████| 983kB 6.5MB/s 
     |████████████████████████████████| 798kB 15.6MB/s 
     |████████████████████████████████| 675kB 19.1MB/s 
     |████████████████████████████████| 256kB 16.5MB/s 
     |████████████████████████████████| 890kB 22.1MB/s 
     |████████████████████████████████| 1.1MB 29.5MB/s 
     |████████████████████████████████| 3.8MB 41.9MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=08643e218232baf9ece0dea7d6f2b74c56cde6ca53118ef8093c824255cde0e2
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=5c5b1db4d370e80610981f0e5908768b862706513e244fbd19232508aaaeefc0
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for sqlitedict: 

In [2]:
import pandas as pd

In [36]:
dev = pd.read_csv('/content/dev.csv')
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
train.head()

,Index,Text,Gold,unique_id
0,121.00003,The new funding comes from venture capital fir...,0,5d268d74-215b-49c2-a045-033d0b409eb4
1,238.00078,"Finally, the actors reveal their true identity...",0,0173005b-de6a-42aa-830e-575d1dbc844b
2,262.00064,"Tell me, will it be successful?",0,c557e356-651f-4827-8c4e-47ffe1efddb9
3,221.00017,"Will private insurance be eliminated?, Warren ...",0,c6393017-b7b3-4edc-958e-78af7a709da6
4,326.00022,● Rebooting AI: Building Artificial Intelligen...,0,840890ec-2b7c-4564-a773-0485a51c2440


In [17]:
dev = dev[['Text', 'Gold']].rename(columns={'Text':'text', 'Gold':'label'})
train = train[['Text', 'Gold']].rename(columns={'Text':'text', 'Gold':'label'})
test = test[['Text', 'Gold']].rename(columns={'Text':'text', 'Gold':'label'})

In [18]:
dev.head()

,text,label
0,China faces certain downward pressure due to s...,0
1,"The high of 1565.15 was recorded on October 9,...",0
2,I don't want to make anyone an enemy but I do ...,0
3,Select Bancorp (NASDAQ:SLCT) last posted its q...,0
4,The Dow Jones now stands once again at the thr...,0


In [19]:
dev['label'] = '__label__' + dev['label'].astype(str)
train['label'] = '__label__' + train['label'].astype(str)
test['label'] = '__label__' + test['label'].astype(str)

In [20]:
dev.head()

,text,label
0,China faces certain downward pressure due to s...,__label__0
1,"The high of 1565.15 was recorded on October 9,...",__label__0
2,I don't want to make anyone an enemy but I do ...,__label__0
3,Select Bancorp (NASDAQ:SLCT) last posted its q...,__label__0
4,The Dow Jones now stands once again at the thr...,__label__0


In [21]:
cols = dev.columns.tolist()
cols = cols[-1:] + cols[:-1]
dev = dev[cols]
train = train[cols]
test = test[cols]

In [22]:
dev.head()

,label,text
0,__label__0,China faces certain downward pressure due to s...
1,__label__0,"The high of 1565.15 was recorded on October 9,..."
2,__label__0,I don't want to make anyone an enemy but I do ...
3,__label__0,Select Bancorp (NASDAQ:SLCT) last posted its q...
4,__label__0,The Dow Jones now stands once again at the thr...


In [23]:
dev.to_csv('dev_flair.csv', sep='\t', index = False, header = False)
test.to_csv('test_flair.csv', sep='\t', index = False, header = False)
train.to_csv('train_flair.csv', sep='\t', index = False, header = False)

In [34]:
from torch.optim.adam import Adam
from flair.data import Corpus
from flair.datasets import TREC_6
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

In [37]:
data_folder = '/content'

In [38]:
column_name_map = {1: "text", 2: "label_causal"}

In [40]:
# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter=',',    # tab-separated files
) 

2020-06-21 21:52:02,071 Reading data from /content
2020-06-21 21:52:02,073 Train: /content/train.csv
2020-06-21 21:52:02,075 Dev: /content/dev.csv
2020-06-21 21:52:02,077 Test: /content/test.csv


In [41]:
# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

# 3. initialize transformer document embeddings (many models are available)
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

# 4. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

# 5. initialize the text classifier trainer with Adam optimizer
trainer = ModelTrainer(classifier, corpus, optimizer=Adam)

# 6. start the training
trainer.train('/content/fincausal_classifier_95_1',
              learning_rate=3e-5, # use very small learning rate
              mini_batch_size=16,
              mini_batch_chunk_size=4, # optionally set this if transformer is too much for your machine
              max_epochs=5, # terminate after 5 epochs
              )

2020-06-21 21:53:25,286 Computing label dictionary. Progress:


100%|██████████| 20069/20069 [00:25<00:00, 792.21it/s]

2020-06-21 21:53:50,857 [b'0', b'1']



2020-06-21 21:54:02,631 ----------------------------------------------------------------------------------------------------
2020-06-21 21:54:02,634 Model: "TextClassifier(
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(i

{'dev_loss_history': [0.11636637151241302,
  0.10893271118402481,
  0.1486188918352127,
  0.22140933573246002,
  0.2202185094356537],
 'dev_score_history': [0.9496221662468514,
  0.9677581863979848,
  0.9642317380352645,
  0.9566750629722922,
  0.9657430730478589],
 'test_score': 0.956482320942883,
 'train_loss_history': [0.1374068212100941,
  0.0680471262077475,
  0.03647703042860336,
  0.019386799739850493,
  0.014220951884869387]}

In [42]:
from flair.models import TextClassifier
from flair.data import Sentence

In [44]:
classifier = TextClassifier.load('/content/drive/My Drive/fincausal_classifier_95_1/best-model.pt')

2020-06-21 22:56:33,950 loading file /content/drive/My Drive/fincausal_classifier_95_1/best-model.pt


In [65]:
def flair_predict(text: str):
  text = Sentence(text)
  classifier.predict(text)
  return text.labels[0].value

In [66]:
test['preds'] = test.Text.apply(flair_predict)
test.head()

,Index,Text,Gold,unique_id,preds
0,352.00039,One bit of advice Orton had for young finance ...,0,f56182ea-5095-4934-889c-f18c1a26134c,0
1,8.00001,President Muhammadu Buhari has disclosed that ...,1,06cc12f1-1df2-43ef-898a-85ebb29bdede,0
2,53.00010,Advent of compressed exhaust systems based on ...,0,a2705c77-271b-4870-b948-8b688657f39c,0
3,533.00022,Catholic leaders have also suggested the UK go...,0,7966c427-9695-4e2f-a098-a5d385d6e0c4,0
4,316.00040,(NasdaqGS:CME) is 66. A company with a value o...,0,14e3e218-9f6b-4949-96c7-62ca68598d55,0


In [70]:
test['Gold'] = test['Gold'].astype(int)
test['preds'] = test['preds'].astype(int)

In [71]:
test.preds.unique()

array([0, 1])

In [68]:
from sklearn.metrics import f1_score

In [72]:
f1_score(test['Gold'], test['preds'], average='weighted')

0.9514955545252303

# New section